In [2]:
!pip install -q python-igraph scikit-learn

In [3]:
import torch
from torch.utils.data import Subset
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

# Define the transformation pipeline:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (x > 0.5).float()),  # Binarize the image
    transforms.Lambda(lambda x: x.view(-1))           # Flatten into a 784-dim vector
])

# Load the training set (set download=True if running for the first time)
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Get indices for images where the label is 1
indices = (mnist_train.targets == 1).nonzero().squeeze()

# Create a subset containing only the '1's
mnist_train_ones = Subset(mnist_train, indices)

print(f"Total number of '1' images in the training set: {len(mnist_train_ones)}")

# Stack all 784-dim vectors from the filtered dataset
all_vectors = torch.stack([img for img, _ in mnist_train_ones])
unique_vectors = torch.unique(all_vectors, dim=0)

print(f"Total images in mnist_train_ones: {all_vectors.shape[0]}")
print(f"Unique images: {unique_vectors.shape[0]}")

if all_vectors.shape[0] == unique_vectors.shape[0]:
    print("All 784-dimensional vectors are unique.")
else:
    print("There are duplicates in the 784-dimensional vectors.")



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 479kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.44MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.87MB/s]


Total number of '1' images in the training set: 6742
Total images in mnist_train_ones: 6742
Unique images: 6726
There are duplicates in the 784-dimensional vectors.


In [4]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import igraph as ig

def visualize_threshold_graph_igraph_weighted(unique_vectors, distance_threshold):
    """
    Builds a weighted graph with python-igraph where each unique vector is a node
    and an edge is added between nodes if their Euclidean distance is <= distance_threshold.
    The edge is weighted by the real Euclidean distance between the nodes.

    Args:
        unique_vectors (torch.Tensor or np.array): Array/tensor of shape (n, 784) containing unique images.
        distance_threshold (float): Maximum Euclidean distance for adding an edge between two nodes.

    Returns:
        g (igraph.Graph): The constructed weighted graph.
    """
    # Convert tensor to numpy array if necessary.
    if hasattr(unique_vectors, 'numpy'):
        X = unique_vectors.numpy()
    else:
        X = unique_vectors

    n = X.shape[0]
    print(f"Building weighted graph for {n} nodes...")

    # Use NearestNeighbors to find all pairs within the distance threshold.
    nbrs = NearestNeighbors(radius=distance_threshold, algorithm='auto').fit(X)
    distances, indices = nbrs.radius_neighbors(X)

    # Build lists of edges and corresponding weights.
    edge_list = []
    weights = []
    for i, (dists, neigh) in enumerate(zip(distances, indices)):
        # For each neighbor j of node i, add edge only if i < j to avoid duplicates.
        for d, j in zip(dists, neigh):
            if i < j:
                edge_list.append((i, j))
                weights.append(d)

    # Create an undirected graph using python-igraph.
    g = ig.Graph(n=n, edges=edge_list, directed=False)
    # Set the edge weight attribute.
    g.es['weight'] = weights

    print(f"Weighted graph built with {len(edge_list)} edges.")
    return g

# Example usage:
g = visualize_threshold_graph_igraph_weighted(unique_vectors, distance_threshold=6)


Building weighted graph for 6726 nodes...
Weighted graph built with 4681902 edges.


In [8]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

# Assume:
#   - g is your igraph Graph from the previous cell.
#   - unique_vectors is a torch.Tensor of shape (n, 784) where n is the number of nodes in g.

# Compute connected components of the graph.
clusters = g.connected_components()
# Find the index (label) of the largest component.
largest_cluster_index = np.argmax(clusters.sizes())
# Get the indices of nodes belonging to the largest component.
vertex_indices = [i for i, comp in enumerate(clusters.membership) if comp == largest_cluster_index]

print(f"Largest connected component has {len(vertex_indices)} nodes.")

# Convert the vertex indices list to a torch tensor.
vertex_indices_tensor = torch.tensor(vertex_indices, dtype=torch.long)

# Use these indices to select the corresponding vectors.
largest_component_vectors = unique_vectors[vertex_indices_tensor]
print("Shape of largest component vectors:", largest_component_vectors.shape)

# Create a TensorDataset with these vectors.
unique_dataset = TensorDataset(largest_component_vectors)
print("TensorDataset created with largest connected component vectors.")


Largest connected component has 6631 nodes.
Shape of largest component vectors: torch.Size([6631, 784])
TensorDataset created with largest connected component vectors.


In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

# Define a simple autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, latent_dim=32):
        super(Autoencoder, self).__init__()
        # Encoder: 784 -> 256 -> 128 -> latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim),
            nn.ReLU()
        )
        # Decoder: latent_dim -> 128 -> 256 -> 784
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()  # Output between 0 and 1
        )

    def forward(self, x):
        z = self.encoder(x)
        reconstructed = self.decoder(z)
        return reconstructed

    def get_latent(self, x):
        return self.encoder(x)

# Training parameters
batch_size = 256
num_epochs = 50
learning_rate = 0.001
latent_dim = 32

# Create DataLoader for the unique dataset
unique_loader = DataLoader(unique_dataset, batch_size=batch_size, shuffle=True)

# Set up device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder(latent_dim=latent_dim).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()  # Use binary cross entropy since inputs are binary
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop for the autoencoder on unique vectors
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in unique_loader:
        images = batch[0].to(device)  # images shape: [batch_size, 784]
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(unique_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Extract latent representations for analysis or visualization
model.eval()
with torch.no_grad():
    sample_batch = next(iter(unique_loader))[0].to(device)
    latent_codes = model.get_latent(sample_batch)
    print("Latent representations shape:", latent_codes.shape)


Epoch [1/50], Loss: 0.4346
Epoch [2/50], Loss: 0.1282
Epoch [3/50], Loss: 0.1161
Epoch [4/50], Loss: 0.1112
Epoch [5/50], Loss: 0.0946
Epoch [6/50], Loss: 0.0728
Epoch [7/50], Loss: 0.0641
Epoch [8/50], Loss: 0.0560
Epoch [9/50], Loss: 0.0521
Epoch [10/50], Loss: 0.0506
Epoch [11/50], Loss: 0.0498
Epoch [12/50], Loss: 0.0491
Epoch [13/50], Loss: 0.0486
Epoch [14/50], Loss: 0.0482
Epoch [15/50], Loss: 0.0479
Epoch [16/50], Loss: 0.0475
Epoch [17/50], Loss: 0.0466
Epoch [18/50], Loss: 0.0429
Epoch [19/50], Loss: 0.0393
Epoch [20/50], Loss: 0.0380
Epoch [21/50], Loss: 0.0370
Epoch [22/50], Loss: 0.0360
Epoch [23/50], Loss: 0.0350
Epoch [24/50], Loss: 0.0339
Epoch [25/50], Loss: 0.0326
Epoch [26/50], Loss: 0.0318
Epoch [27/50], Loss: 0.0311
Epoch [28/50], Loss: 0.0305
Epoch [29/50], Loss: 0.0301
Epoch [30/50], Loss: 0.0295
Epoch [31/50], Loss: 0.0290
Epoch [32/50], Loss: 0.0284
Epoch [33/50], Loss: 0.0280
Epoch [34/50], Loss: 0.0275
Epoch [35/50], Loss: 0.0269
Epoch [36/50], Loss: 0.0265
E

In [10]:
import torch
import numpy as np

def compare_distances(model, dataset, device, sample_size=50):
    # Randomly sample sample_size images from the dataset
    indices = torch.randperm(len(dataset))[:sample_size]
    sample = torch.stack([dataset[i][0] for i in indices]).to(device)  # [sample_size, 784]

    # Get latent representation for the sample
    model.eval()
    with torch.no_grad():
        latent = model.get_latent(sample)  # [sample_size, latent_dim]

    # Compute pairwise Euclidean distances in the original and latent spaces
    dist_original = torch.cdist(sample, sample, p=2)
    dist_latent = torch.cdist(latent, latent, p=2)

    # Compute correlation between the flattened distance matrices
    corr = np.corrcoef(dist_original.cpu().numpy().flatten(),
                       dist_latent.cpu().numpy().flatten())[0, 1]
    print("Correlation (Euclidean distances) between original and latent spaces:", corr)

    return dist_original, dist_latent


In [53]:
dist_original, dist_latent = compare_distances(model, unique_dataset, device, sample_size=100)

Correlation (Euclidean distances) between original and latent spaces: 0.8464987402608786


In [64]:
# Compute latent representations for each unique vector using the autoencoder
model.eval()
with torch.no_grad():
    latent_vectors = model.get_latent(unique_vectors)

print("Latent representations shape:", latent_vectors.shape)

# Build a weighted graph from the latent representations using the pre-defined function
g_latent = visualize_threshold_graph_igraph_weighted(latent_vectors, distance_threshold=6)

# Find connected components in the latent graph
clusters_latent = g_latent.connected_components()
largest_component_index = np.argmax(clusters_latent.sizes())
largest_component_nodes = clusters_latent[largest_component_index]

# Create a subgraph of the largest connected component
subgraph = g_latent.subgraph(largest_component_nodes)

# Print the number of nodes and edges in the largest connected component
print("Largest connected component:")
print("Number of nodes:", subgraph.vcount())
print("Number of edges:", subgraph.ecount())



Latent representations shape: torch.Size([6726, 32])
Building weighted graph for 6726 nodes...
Weighted graph built with 107336 edges.
Largest connected component:
Number of nodes: 6464
Number of edges: 107242


In [65]:
def compare_graph_connectivity(g_orig, g_latent):
    """
    For each node, print the neighbors in the original and latent graphs,
    and compute the symmetric difference between the two neighbor sets.
    Then, identify nodes with the greatest differences.
    """
    differences = {}
    for node in range(g_orig.vcount()):
        # Get neighbor sets from both graphs
        neighbors_orig = set(g_orig.neighbors(node))
        neighbors_latent = set(g_latent.neighbors(node))
        # Compute symmetric difference
        diff = neighbors_orig.symmetric_difference(neighbors_latent)
        differences[node] = diff

        # print(f"Node {node}:")
        # print(f"  Original neighbors: {sorted(neighbors_orig)}")
        # print(f"  Latent neighbors:   {sorted(neighbors_latent)}")
        # print(f"  Difference:         {sorted(diff)}\n")

    # Identify nodes with the greatest differences
    diff_counts = {node: len(diff) for node, diff in differences.items()}
    max_diff_nodes = sorted(diff_counts.items(), key=lambda x: x[1], reverse=True)[:5]

    print("Nodes with greatest connectivity differences:")
    for node, count in max_diff_nodes:
        print(f"  Node {node} has {count} differing connections.")

    return differences

# Compare connectivity between the original graph 'g' and the latent graph 'g_latent'
diffs = compare_graph_connectivity(g, g_latent)

# Print total edge counts for both graphs
print(f"\nTotal edges in original graph: {g.ecount()}")
print(f"Total edges in latent graph:   {g_latent.ecount()}")


Nodes with greatest connectivity differences:
  Node 1352 has 2518 differing connections.
  Node 5131 has 2507 differing connections.
  Node 3930 has 2500 differing connections.
  Node 4552 has 2490 differing connections.
  Node 3978 has 2483 differing connections.

Total edges in original graph: 4681902
Total edges in latent graph:   107336
